In [4]:
import pandas as pd

# Wczytanie pliku CSV
file_path = "filtered_mbsa_full.csv"  # Ścieżka do pliku
data = pd.read_csv(file_path)

# Wyświetlenie podstawowych informacji o zbiorze danych
print("Podstawowe informacje o zbiorze danych:")
print(data.info())  # Informacje o kolumnach, typach danych i brakujących wartościach

print("\nPierwsze 5 wierszy zbioru danych:")
print(data.head())  # Podgląd pierwszych 5 wierszy

print("\nStatystyki opisowe zbioru danych:")
print(data.describe())  # Podstawowe statystyki opisowe dla danych numerycznych

# Wyświetlenie liczby wierszy i kolumn
print(f"\nLiczba wierszy: {data.shape[0]}")
print(f"Liczba kolumn: {data.shape[1]}")


Podstawowe informacje o zbiorze danych:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14050565 entries, 0 to 14050564
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   Date       object
 1   text       object
 2   Sentiment  object
dtypes: object(3)
memory usage: 321.6+ MB
None

Pierwsze 5 wierszy zbioru danych:
         Date                                               text Sentiment
0  2019-05-27  Cardano: Digitize Currencies; EOS https://t.co...  Positive
1  2019-05-27  Another Test tweet that wasn't caught in the s...  Positive
2  2019-05-27  Current Crypto Prices! \n\nBTC: $8721.99 USD\n...  Positive
3  2019-05-27  Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...  Positive
4  2019-05-27  @nwoodfine We have been building on the real #...  Positive

Statystyki opisowe zbioru danych:
              Date                                               text  \
count     14050565                                           14050565   
unique        1901

In [5]:
# Usuwanie duplikatów
data = data.drop_duplicates()
# Usuwanie wierszy z brakującymi wartościami
data = data.dropna()

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Usunięcie neutralnych przykładów
filtered_data = data[data["Sentiment"] != "Neutral"]

# Pobranie pierwszego miliona przykładów
filtered_data = filtered_data.head(1000000)

# Zapisanie przetworzonego zbioru do nowego pliku CSV
filtered_data.to_csv("filtered_mbsa_first_million.csv", index=False)

# Inicjalizacja analizatora sentymentu
analyzer = SentimentIntensityAnalyzer()

# Funkcja do klasyfikacji komentarzy
def classify_sentiment(text):
    scores = analyzer.polarity_scores(text)
    compound_score = scores['compound']
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return None

# Przetwarzanie danych i porównanie wyników
correct = 0
total = 0

for index, row in filtered_data.iterrows():
    predicted_sentiment = classify_sentiment(row["text"])
    if predicted_sentiment is not None:  # Pomijanie neutralnych wyników
        total += 1
        if predicted_sentiment == row["Sentiment"]:
            correct += 1

# Obliczenie dokładności
accuracy = correct / total if total > 0 else 0

# Wynik
print(f"Dokładność analizatora: {accuracy:.2%}")

Dokładność analizatora: 55.06%


In [6]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# (1) Przykładowe wczytanie / przefiltrowanie danych
# ---------------------------------------------------
filtered_data = data[data["Sentiment"] != "Neutral"]


# (2) Inicjalizacja analizatora sentymentu
analyzer = SentimentIntensityAnalyzer()

# (3) Funkcja do klasyfikacji komentarzy
def classify_sentiment(text):
    scores = analyzer.polarity_scores(text)
    compound_score = scores['compound']
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return None

# (4) Listy do przechowywania etykiet rzeczywistych i przewidywanych
y_true = []
y_pred = []

# (5) Klasyfikacja i gromadzenie danych do obliczania metryk
for index, row in filtered_data.iterrows():
    predicted_sentiment = classify_sentiment(row["text"])
    # Ignorujemy przypadki, gdy model zwraca None (neutralne)
    if predicted_sentiment is not None:
        y_true.append(row["Sentiment"])
        y_pred.append(predicted_sentiment)

# (6) Obliczanie macierzy pomyłek oraz metryk
# ------------------------------------------
# labels określa kolejność etykiet w macierzy [Negative, Positive]
cm = confusion_matrix(y_true, y_pred, labels=["Negative", "Positive"])
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, labels=["Negative", "Positive"], pos_label="Positive")
recall = recall_score(y_true, y_pred, labels=["Negative", "Positive"], pos_label="Positive")
f1 = f1_score(y_true, y_pred, labels=["Negative", "Positive"], pos_label="Positive")

# (7) Wyświetlenie wyników
print("Macierz pomyłek (Confusion Matrix):")
print(cm)
print(f"Accuracy:  {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall:    {recall:.2%}")
print(f"F1-score:  {f1:.2%}")


Macierz pomyłek (Confusion Matrix):
[[1003615 2610275]
 [ 968795 2531500]]
Accuracy:  49.69%
Precision: 49.23%
Recall:    72.32%
F1-score:  58.59%


In [3]:
print("Unikalne w y_true:", set(y_true))
print("Unikalne w y_pred:", set(y_pred))

Unikalne w y_true: {nan, 'Negative', 'Positive'}
Unikalne w y_pred: {'Negative', 'Positive'}
